# Milestone 1: Week 1-2 — Smart Energy Analysis
 
## Module 1: Data Collection and Understanding
- **Define project scope and functional objectives for smart energy analysis.**
- **Collect and structure the SmartHome Energy Monitoring Dataset.**
- **Verify data integrity, handle missing timestamps, and perform exploratory analysis.**
- **Organize energy readings by device, room, and timestamp.**
 
## Module 2: Data Cleaning and Preprocessing
- **Handle missing values and outliers in power consumption readings.**
- **Convert timestamps to datetime format and resample data (hourly/daily).**
- **Normalize or scale energy values for model compatibility.**
- **Split dataset into training, validation, and testing sets.**
 
---
 
This notebook implements the highest level of these tasks for Milestone 1, with all outputs and images saved for reporting and reproducibility.

## NOTE:
This notebook submission is limited to the following Milestone 1 tasks only:
- Data Collection and Understanding (Module 1)
- Data Cleaning and Preprocessing (Module 2)
All other modules, advanced modeling, or extra analyses are excluded from this milestone.

In [1]:
# Data Manipulation & Analysis
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Deep Learning (LSTM)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("✅ All libraries imported successfully!")
print(f"📦 TensorFlow version: {tf.__version__}")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

✅ All libraries imported successfully!
📦 TensorFlow version: 2.20.0
📦 Pandas version: 2.3.3
📦 NumPy version: 2.2.6


## 2️⃣ Load and Parse Dataset

The dataset uses:
- Semicolon (`;`) as delimiter
- Missing values marked as `?`
- European decimal format (comma as separator) - though this dataset uses period

In [2]:
# Load the dataset
# The dataset uses semicolon as separator and '?' for missing values
df = pd.read_csv('household_power_consumption.txt', 
                 sep=';', 
                 na_values=['?', ''],
                 low_memory=False)

# Display basic information
print("=" * 60)
print("📊 DATASET LOADED SUCCESSFULLY!")
print("=" * 60)
print(f"\n📏 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n📋 Column Names:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i}. {col}")

# Display first few rows
print("\n🔍 First 5 rows:")
df.head()

📊 DATASET LOADED SUCCESSFULLY!

📏 Shape: 2,075,259 rows × 9 columns
💾 Memory Usage: 338.33 MB

📋 Column Names:
   1. Date
   2. Time
   3. Global_active_power
   4. Global_reactive_power
   5. Voltage
   6. Global_intensity
   7. Sub_metering_1
   8. Sub_metering_2
   9. Sub_metering_3

🔍 First 5 rows:


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [3]:
# Check data types and info
print("📊 Dataset Information:")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("📈 Statistical Summary:")
print("=" * 60)
df.describe()

📊 Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    float64
 3   Global_reactive_power  float64
 4   Voltage                float64
 5   Global_intensity       float64
 6   Sub_metering_1         float64
 7   Sub_metering_2         float64
 8   Sub_metering_3         float64
dtypes: float64(7), object(2)
memory usage: 142.5+ MB

📈 Statistical Summary:


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06
mean,1.091615e+00,1.237145e-01,2.408399e+02,4.627759e+00,1.121923e+00,1.298520e+00,6.458447e+00
std,1.057294e+00,1.127220e-01,3.239987e+00,4.444396e+00,6.153031e+00,5.822026e+00,8.437154e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.020000e-01,1.000000e-01,2.410100e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.528000e+00,1.940000e-01,2.428900e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01


## 3️⃣ Data Cleaning and Missing Value Treatment

### Key Preprocessing Steps:
1. Convert all numeric columns to proper float type
2. Handle missing values using forward-fill and interpolation
3. Create proper DateTime index

In [4]:
# Check missing values
print("🔍 Missing Values Analysis:")
print("=" * 60)
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage (%)': missing_percent
})
print(missing_df)
print(f"\n📊 Total missing values: {df.isnull().sum().sum():,}")
print(f"📊 Percentage of data with missing values: {(df.isnull().any(axis=1).sum() / len(df) * 100):.2f}%")

🔍 Missing Values Analysis:
                       Missing Count  Percentage (%)
Date                               0            0.00
Time                               0            0.00
Global_active_power            25979            1.25
Global_reactive_power          25979            1.25
Voltage                        25979            1.25
Global_intensity               25979            1.25
Sub_metering_1                 25979            1.25
Sub_metering_2                 25979            1.25
Sub_metering_3                 25979            1.25

📊 Total missing values: 181,853
📊 Percentage of data with missing values: 1.25%


In [5]:
# Convert numeric columns to proper types
numeric_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Create DateTime column
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')

# Set DateTime as index
df.set_index('DateTime', inplace=True)

# Drop original Date and Time columns
df.drop(['Date', 'Time'], axis=1, inplace=True)

print("✅ DateTime index created!")
print(f"📅 Date Range: {df.index.min()} to {df.index.max()}")
print(f"📆 Total Duration: {(df.index.max() - df.index.min()).days} days")
df.head()

✅ DateTime index created!
📅 Date Range: 2006-12-16 17:24:00 to 2010-11-26 21:02:00
📆 Total Duration: 1441 days


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
DateTime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [6]:
# Handle missing values using forward fill and interpolation
print("🔧 Handling Missing Values...")
print(f"   Before: {df.isnull().sum().sum():,} missing values")

# Forward fill first (for time-series continuity)
df.fillna(method='ffill', inplace=True)

# Then backward fill for any remaining at the start
df.fillna(method='bfill', inplace=True)

# Verify
print(f"   After: {df.isnull().sum().sum():,} missing values")
print("\n✅ Missing values handled successfully!")

# Display cleaned data info
print("\n📊 Cleaned Dataset Summary:")
df.describe().round(3)

🔧 Handling Missing Values...
   Before: 181,853 missing values
   After: 0 missing values

✅ Missing values handled successfully!

📊 Cleaned Dataset Summary:


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2075259.000,2075259.000,2075259.000,2075259.000,2075259.000,2075259.000,2075259.000
mean,1.086,0.123,240.842,4.604,1.111,1.288,6.417
std,1.053,0.113,3.236,4.427,6.116,5.787,8.420
min,0.076,0.000,223.200,0.200,0.000,0.000,0.000
25%,0.308,0.048,239.000,1.400,0.000,0.000,0.000
50%,0.598,0.100,241.020,2.600,0.000,0.000,1.000
75%,1.524,0.194,242.870,6.400,0.000,1.000,17.000
max,11.122,1.390,254.150,48.400,88.000,80.000,31.000


## 4️⃣ DateTime Feature Engineering

Extracting temporal features for pattern analysis:
- Hour of day (0-23)
- Day of week (0-6, Monday=0)
- Month (1-12)
- Year
- Weekend indicator
- Season

In [7]:
# Extract temporal features
df['Hour'] = df.index.hour
df['DayOfWeek'] = df.index.dayofweek
df['Month'] = df.index.month
df['Year'] = df.index.year
df['Day'] = df.index.day
df['Quarter'] = df.index.quarter
df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)

# Create Season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['Season'] = df['Month'].apply(get_season)

# Day period (Morning, Afternoon, Evening, Night)
def get_day_period(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['DayPeriod'] = df['Hour'].apply(get_day_period)

print("✅ Temporal features created!")
print("\n📊 Sample of engineered features:")
df[['Global_active_power', 'Hour', 'DayOfWeek', 'Month', 'Year', 'IsWeekend', 'Season', 'DayPeriod']].head(10)

✅ Temporal features created!

📊 Sample of engineered features:


,Global_active_power,Hour,DayOfWeek,Month,Year,IsWeekend,Season,DayPeriod
DateTime,,,,,,,,
2006-12-16 17:24:00,4.216,17,5,12,2006,1,Winter,Evening
2006-12-16 17:25:00,5.360,17,5,12,2006,1,Winter,Evening
2006-12-16 17:26:00,5.374,17,5,12,2006,1,Winter,Evening
2006-12-16 17:27:00,5.388,17,5,12,2006,1,Winter,Evening
2006-12-16 17:28:00,3.666,17,5,12,2006,1,Winter,Evening
2006-12-16 17:29:00,3.520,17,5,12,2006,1,Winter,Evening
2006-12-16 17:30:00,3.702,17,5,12,2006,1,Winter,Evening
2006-12-16 17:31:00,3.700,17,5,12,2006,1,Winter,Evening
2006-12-16 17:32:00,3.668,17,5,12,2006,1,Winter,Evening


## End of Milestone 1 Submission
This is the end of the Milestone 1 (Week 1-2) submission. All subsequent content, including advanced modeling, feature engineering, or analyses beyond the scope of Modules 1 and 2, has been intentionally removed for this milestone.